In [ ]:
import sys
import numpy as np
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/utils')
import eval_utils
import cryptoshow_utils
import vis_utils

CBS_DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/data-extraction/cryptobench-clustered-binding-sites.csv'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/visualizations/predictions'

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=CBS_DATA_PATH)
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/cryptobench-with-clustering', protein_ids=binding_residues_mmcifed.keys())
smoothed_model_predictions_mmcifed = {pid: np.array(preds) for pid, (_, preds) in model_predictions_mmcifed.items()}
model_predictions_mmcifed = {pid: [np.array(i) for i in preds] for pid, (preds, _) in model_predictions_mmcifed.items()}

# map binding residues to auth labeling
binding_residues = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in binding_residues_mmcifed.items()}
smoothed_model_predictions = {protein_id: cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for protein_id, pocket in smoothed_model_predictions_mmcifed.items()}
model_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}

In [ ]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd

CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'

In [13]:
skip = True
for protein_id in binding_residues.keys():
    if skip:
        if protein_id == '7o1iA':
            skip = False
        else:
            continue
    if protein_id not in model_predictions or protein_id not in smoothed_model_predictions:
        continue
    this_model_predictions = np.concatenate((model_predictions[protein_id]), axis=0)
    this_smoothed_model_predictions = smoothed_model_predictions[protein_id]
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)

    if len(this_model_predictions) != 0:

        cmd.color('blue', vis_utils.generate_pymol_algebra_selection(protein_id, this_model_predictions))
        if len(this_smoothed_model_predictions) > 0:
            cmd.color('grey', vis_utils.generate_pymol_algebra_selection(protein_id, this_smoothed_model_predictions))
    
    cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
PyMOL>color green, resi 25+63+65+112+300+304+305
 Executive: Colored 57 atoms.
PyMOL>color grey, resi 25+63+65+112+300+304+305
 Executive: Colored 57 atoms.
PyMOL>color green, resi 25+63+65+112+300+304+305
 Executive: Colored 57 atoms.
PyMOL>color grey, resi 25+63+65+112+300+304+305
 Executive: Colored 57 atoms.
PyMOL>set opaque_background, 0
 Setting: opaque_background set to off.
PyMOL>png /home/vit/Downloads/7o1iA.png, 0, 0, -1, ray=1
 Ray: render time: 2.93 sec. = 1228.6 frames/hour (9.32 sec. accum.).
 ScenePNG: wrote 640x508 pixel image to file "/home/vit/Downloads/7o1iA.png".
PyMOL>set opaque_background, 0
 Setting: opaque_background set to off.
PyMOL>png /home/vit/Downloads/7o1iA-no-smooth.png, 0, 0, -1, ray=1
 Ray: render time: 3.13 sec. = 1151.4 frames/hour (12.45 sec. accum.).
 ScenePNG: wrote 640x508 pixel image to file "/home/vit/Downloads/7o1iA-no-smooth.p